# Inventory Modeling and Analysis Made Easy with Supplychainpy
** *by Kevin Fasusi* **

This workbook assumes some familiarity and proficiency in programming with Python. Understanding list comprehensions, conditional logic and functions are an essential prerequisite for continuing with this workbook.

Typically, inventory analysis using Excel requires several formulas, manual processes, possibly some pivot tables and in some cases VBA to achieve. Using the supplychainpy library can reduce the time taken and effort made for the same analysis.

In [ ]:
from supplychainpy.model_inventory import analyse
from decimal import Decimal
from supplychainpy.sample_data.config import ABS_FILE_PATH

The first two imports are mandatory; the second import is necessary for using the sample data in the `supplychainpy` library. When working with a different file, supply the file path to the `file_path` parameter. The data provided for analysis can be from a `CSV` or a database ETL process.

The sample data is a `CSV` formatted file:

In [ ]:
with open(ABS_FILE_PATH['COMPLETE_CSV_SM'],'r') as raw_data:
    for line in raw_data:
        print(line)

It is probable that getting the data into this format will require 'extracting' from a database and 'transforming' data before 'loading' into the `analyse` function. To perform the ETL, an ORM like SQLAlchemy or appropriate DB-API for the database in question may be of use. Supplychainpy works with Pandas, so performing the transformations using Pandas may be an idea. The `DataFrame` or file passed as an argument must be identical to the format above (future versions of supplychainpy will be more lenient and attempt to identify if a minimum requirement is satisfied).

The ETL process is not covered in this workbook but on the 'roadmap' for `supplychainpy` is the automation of this process. 

So now that we have the data in the correct format, we can proceed with the analysis.

In [ ]:
#%%timeit
analysed_inventory_profile= analyse(file_path=ABS_FILE_PATH['COMPLETE_CSV_SM'],
                                                             z_value=Decimal(1.28),
                                                             reorder_cost=Decimal(400),
                                                             file_type='csv')


The variable `analysed_inventory_profile` now contains a collection (list) of `UncertainDemand` objects (one per SKU). Each object contains the analysis for each SKU. The analysis include the following:

- safety stock
- total_orders
- standard_deviation
- quantity_on_hand': '1003
- economic_order_variable_cost
- sku
- economic_order_quantity
- unit_cost
- demand_variability
- average_orders
- excess_stock
- currency
- ABC_XYZ_Classification
- shortages
- reorder_level
- revenue 
- reorder_quantity 
- safety_stock
- orders

The listed summary items can be retrieved by calling the method `orders_summary()` on each object. The quickest way to do this is with a list comprehension.

In [ ]:
analysis_summary = [demand.orders_summary() for demand in analysed_inventory_profile]
print(analysis_summary)

For the intrepid reader who did not heed the warning about the prerequisites and is now scratching their head wondering "what manner of black magic is this," here is a quick overview on list comprehensions. In short, the above code is similar to the code below:

In [ ]:
analysis_summary =[]
for demand in analysed_inventory_profile:
    analysis_summary.append(demand.orders_summary())

The former is much more readable.

## Exploring the results

To make sense of the results, we can filter our analysis using standard Python scripting techniques. For example to retrieve the whole `summary` for the SKU `KR202-209` we can do something like this:

In [ ]:
%%timeit
sku_summary = [demand.orders_summary() for demand in analysed_inventory_profile if demand.orders_summary().get('sku')== 'KR202-209']


The inventory classification ABC XYZ denotes the SKUs contribution to revenue and demand volatility. `AX` SKUs typically exhibit steady demand and contribute 80% of the value of income for the period under analysis.

As a more traditional way of grouping SKUs by behaviour, it is also likely to be used for generating inventory policies and for further exploration of the stock profile. To retrieve all the summaries for a particular classification, we could do something like this:

In [ ]:
ay_classification_summary = [demand.orders_summary() for demand in analysed_inventory_profile if demand.orders_summary().get('ABC_XYZ_Classification')== 'AY']
print(ay_classification_summary)

Using a built-in feature of the library provides a quicker way to filter the results. For example a quciker way to filter for SKU `KR202-209`, is through the use of `Inventory` class in the `summarise` module.

In [ ]:
from supplychainpy.inventory.summarise import Inventory
filtered_summary = Inventory(analysed_inventory_profile)

In [ ]:
%%timeit
sku_summary = [summary for summary in filtered_summary.describe_sku('KR202-209')]
#print(sku_summary)

Using the import Inventory specifically built to filter the analysis is faster and syntactically cleaner for easier to read and understand the code. The `Inventory` summary class also provides a more detailed overview of the SKU with additional KPIs and metric in the context of the whole stock profile. The `orders_summary` method ranks and performs some comparative analysis for more insight.

The descriptive summary includes:

- shortage_rank
- min_orders
- excess_units
- revenue_rank
- excess_rank
- average_orders
- gross_profit_margin
- markup_percentage
- max_order
- shortage_cost
- quantity_on_hand
- inventory_turns
- sku_id
- retail_price
- revenue_rank 
- shortage_units
- unit_cost
- classification
- safety_stock_cost
- safety_stock_units
- safety_stock_rank
- percentage_contribution_revenue
- gross_profit_margin
- shortage_rank
- inventory_traffic_light
- unit_cost_rank
- excess_cost
- excess_units
- markup_percentage
- revenue

Further summaries can be retrieved, for instance, reviews at the inventory classification level of detail can be quite useful when exploring inventory policies:

In [ ]:
classification_summary =  [summary for summary in filtered_summary.abc_xyz_summary(classification=('AY',), category=('revenue',))]
print(classification_summary)

Now we know the total revenue generated by the `AY` SKU class. There is another, slightly more fun way to arrive at this number using `Dash` but more on that later.

In [ ]:
top_10_safety_stock_skus =  [summary.get('sku')for summary in filtered_summary.rank_summary(attribute='safety_stock', count=10)]
print(top_10_safety_stock_skus)

Let us add the `safety_stock` and create a tuple to see that the results explicitly.

In [ ]:
top_10_safety_stock_values =  [(summary.get('sku'), summary.get('safety_stock'))for summary in filtered_summary.rank_summary(attribute='safety_stock', count=10)]
print(top_10_safety_stock_values)

We can then pass back the list of `top_10_safety_stock_skus` back into the inventory filter and get their breakdown.

In [ ]:
top_10_safety_stock_summary = [summary for summary in filtered_summary.describe_sku(*top_10_safety_stock_skus)]
#print(top_10_safety_stock_summary)

We have only covered a few use cases, but we have already achieved a significant amount of analysis with relatively few line of code. The equivalent in Excel would require much more work and many more formulas.